# Experiment 2. Preliminary data import for the main experiment.

In [1]:
import os
import pandas as pd
import numpy as np
import os

In [2]:
### constants
data_folder= 'Experiment 1. Connectedness and Common Region'
buttons= ['J', 'K']
event_code= ['Pressed', 'Released']
observerN= 15

In [3]:
### block conditions for each observers
observer_conditions= pd.read_csv(os.path.join(data_folder, 'Experiment1_BlockList.csv'), sep=';')

## Importing all responses into a single table

In [4]:
### importing all responses into a single table
dfDict= {'ID': [], 'Block':[], 'ConditionLabel': [], 'Time': [], 'Key': [], 'EventLabel': [], 'Target': [], 'Event': []}
for iO in range(1, observerN+1):
    with open(os.path.join(data_folder, 's%02d.txt'%(iO)), 'rt') as response_file:
        current_block= 1
        block_has_started= False
        data_lines= 0
        for current_line in response_file:
            # skipping the "start of the block" line
            if current_line[:len("--Activated")]=="--Activated":
                # advancing block counter
                if block_has_started:
                    ## end of the previous block
                    dfDict['ID'].append('s%02d'%(iO))
                    dfDict['Block'].append(current_block)
                    dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
                    dfDict['Time'].append(possible_end_of_block_time)
                    dfDict['Key'].append(subfields[-1][0])
                    dfDict['EventLabel'].append('Block end')
                    dfDict['Target'].append(-1)
                    dfDict['Event'].append(200)
                    
                    # preparing counters for the next block
                    current_block+= 1
                    block_has_started= False
                    data_lines= 0
                continue
                
            # skipping over an empty line
            if len(current_line.strip())==0:
                continue
            
            subfields= current_line.split()
            # parsing time into seconds
            time_components= subfields[0].split(':') # assuming the hours:minutes:float_seconds
            if len(time_components)==1: # 
                time_components= subfields[0].split('-') # nope, this is hours-minutes-float_seconds
            if len(time_components)==1:
                continue # yet something else, definitely not the relevant response line
            four_time_components= np.concatenate([time_components[0:-1], time_components[-1].split('.')])
            time_in_seconds= np.sum(np.array(four_time_components).astype('float')*np.array([60*60, 60, 1, 1/100.0]))

            if subfields[-1][0] in buttons: # response buttons
                data_lines+= 1
                # adding the next line to the dictionary
                dfDict['ID'].append('s%02d'%(iO))
                dfDict['Block'].append(current_block)
                dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
                dfDict['Time'].append(time_in_seconds)

                # parsing event into target and event type
                dfDict['Key'].append(subfields[-1][0])
                dfDict['EventLabel'].append(subfields[-1].strip()[2:-1])
                dfDict['Target'].append(buttons.index(subfields[-1][0]))
                dfDict['Event'].append(event_code.index(subfields[-1].strip()[2:-1]))
            elif subfields[-1][0] in ['E', 'R']: # special event
                if not block_has_started and subfields[-1].strip()[2:-1]=='Released': 
                    ## block start
                    dfDict['ID'].append('s%02d'%(iO))
                    dfDict['Block'].append(current_block)
                    dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
                    dfDict['Time'].append(time_in_seconds)
                    dfDict['Key'].append(subfields[-1][0])
                    dfDict['EventLabel'].append('Block start')
                    dfDict['Target'].append(-1)
                    dfDict['Event'].append(100)
                    block_has_started= True
                elif subfields[-1].strip()[2:-1]=='Pressed':
                    possible_end_of_block_time= time_in_seconds
            else:
                data_lines+= 1
                
    ## end of the last block
    dfDict['ID'].append('s%02d'%(iO))
    dfDict['Block'].append(current_block)
    dfDict['ConditionLabel'].append(observer_conditions['%d'%(current_block)][observer_conditions.ID==iO].values[0])
    dfDict['Time'].append(possible_end_of_block_time)
    dfDict['Key'].append(subfields[-1][0])
    dfDict['EventLabel'].append('Block end')
    dfDict['Target'].append(-1)
    dfDict['Event'].append(200)

            
responses= pd.DataFrame(dfDict, columns=['ID', 'Block', 'ConditionLabel', 'Time', 'Target', 'Event', 'Key', 'EventLabel'])
responses.to_csv(os.path.join(data_folder, 'Experiment1_Response.csv'), sep=';', index= False)

## Converting key presses into the sequence of subjective states

In [5]:
### turning responses into the sequence of subjective states'
subjective_state_change= pd.DataFrame(columns= ['ID', 'Block', 'BlockDuration', 'ConditionLabel', 'Trial', 'TargetCount', 'Time', 'Duration'])
for current_ID in responses.ID.unique():
    for current_block in responses.Block[responses.ID==current_ID].unique():
        print('\rObserver %s, block %d'%(current_ID, current_block), end=' ')
        iBlockTrials= responses[(responses.ID==current_ID) & (responses.Block==current_block)].index
        
        # figuring out block start-end-duration
        current_block_start= responses.Time[(responses.ID==current_ID) & (responses.Block==current_block) & (responses.EventLabel=='Block start')].values[0]
        current_block_end= responses.Time[(responses.ID==current_ID) & (responses.Block==current_block) & (responses.EventLabel=='Block end')].values[0]
        current_block_duration=current_block_end-current_block_start 

        # block constants
        current_entry= {'ID': current_ID, 
                        'Block': current_block, 
                        'BlockDuration': current_block_duration,
                        'ConditionLabel': responses.ConditionLabel[iBlockTrials[0]]}        
        
        # adding an onset state - all visibile
        current_entry['Trial']= 0
        current_entry['TargetCount']= 2
        current_entry['Duration']= 0
        current_entry['Time']= 0
        subjective_state_change.loc[subjective_state_change.shape[0]] = pd.Series(current_entry)
        
        CurrentTargetCount= 2
        PrevioiusEventTime= 0
        # going through events
        for iTrial, iRow in enumerate(iBlockTrials[1:-1]): # first and last events are labels for start and end of the block
            current_entry['Trial']= iTrial+1
            if responses.Event[iRow]==0:
                CurrentTargetCount-= 1
            else:
                CurrentTargetCount+= 1
            current_entry['TargetCount']= CurrentTargetCount
            current_entry['Time']=  responses.Time[iRow]-current_block_start
            subjective_state_change.set_value(len(subjective_state_change)-1, 'Duration', current_entry['Time']-PrevioiusEventTime)
            if iTrial==len(iBlockTrials)-2: # the last trial
                current_entry['Duration']= current_block_duration-current_entry['Time']
            PrevioiusEventTime= current_entry['Time']
            subjective_state_change.loc[subjective_state_change.shape[0]] = pd.Series(current_entry)
print('\rDone!                      ')
subjective_state_change.to_csv(os.path.join(data_folder, 'Experiment1_SubjectiveStateChange.csv'), sep=';', index= False)

Done!                      


## Preliminary fingers-press asynchrony analysis

In [6]:
### importing all responses into a single table
dfDict= {'ID': [], 'Time': [], 'Key': [], 'EventLabel': [], 'Target': [], 'Event': []}
for iO in range(1, observerN+1):
    with open(os.path.join(data_folder, 's%02d_FOA.txt'%(iO)), 'rt') as response_file:
        for current_line in response_file:
            # skipping the "start of the block" line
            if current_line[:len("--Activated")]=="--Activated":
                continue
                
            subfields= current_line.split()
            if subfields[-1][0] in buttons: # relevant button
                # adding the next line to the dictionary
                dfDict['ID'].append('s%02d'%(iO))

                # parsing time into seconds
                time_components= subfields[0].split(':') # assuming the hours:minutes:float_seconds
                if len(time_components)==1: # 
                    time_components= subfields[0].split('-') # nope, this is hours-minutes-float_seconds
                four_time_components= np.concatenate([time_components[0:-1], time_components[-1].split('.')])
                time_in_seconds= np.sum(np.array(four_time_components).astype('float')*np.array([60*60, 60, 1, 1/100.0]))
                dfDict['Time'].append(time_in_seconds)

                # parsing event into target and event type
                dfDict['Key'].append(subfields[-1][0])
                dfDict['EventLabel'].append(subfields[-1].strip()[2:-1])
                dfDict['Target'].append(buttons.index(subfields[-1][0]))
                dfDict['Event'].append(event_code.index(subfields[-1].strip()[2:-1]))
            
responses= pd.DataFrame(dfDict, columns=['ID', 'Time', 'Target', 'Event', 'Key', 'EventLabel'])
responses.to_csv(os.path.join(data_folder, 'Experiment1_FOA_Response.csv'), sep=';', index= False)

In [7]:
## computing distribution of response times for individual observers
dfDict= {'ID': [], 'Event':[], 'MaxRT':[], 'MeanRT':[], 'MinRT':[], 'StdRT':[]}

### computing response variance for each observer
for current_observer in responses.ID.unique():
    iTrials= responses[responses.ID==current_observer].index
    
    for current_event in range(2):
        # finding relevant double events
        iDoubleEventTrials,= np.where(((responses.Target.values[iTrials[:-1]] != responses.Target.values[iTrials[1:]]) & 
                                      (responses.Event.values[iTrials[:-1]] == responses.Event.values[iTrials[1:]]) &
                                      (responses.Event.values[iTrials[:-1]] == current_event)))
                
        iDoubleEvent= iTrials[iDoubleEventTrials]
        rtDist= responses.Time[iDoubleEvent+1].values-responses.Time[iDoubleEvent].values
        
        # characterizing distribution
        dfDict['ID'].append(current_observer)
        dfDict['Event'].append(current_event)
        dfDict['MaxRT'].append(np.max(rtDist))
        dfDict['MinRT'].append(np.min(rtDist))
        dfDict['MeanRT'].append(np.mean(rtDist))
        dfDict['StdRT'].append(np.std(rtDist))
                                    
ResponseBrackets= pd.DataFrame(dfDict)
ResponseBrackets.to_csv(os.path.join(data_folder, 'Experiment1_FOA_Estimate.csv'), sep=';', index= False)

In [8]:
# throwing away all block start-end events
responses= pd.read_csv(os.path.join(data_folder, 'Experiment1_Response.csv'), sep=';')
FOA= ResponseBrackets
results= responses[responses.Event<100]
results['SimCount']= 0
for current_observer in results.ID.unique():
    print('%s'%(current_observer), end=' ')
    for current_block in results[results.ID==current_observer].Block.unique():
        iBlockTrials= results[(results.ID==current_observer) & (results.Block==current_block)].index
        for iTrial, iRow in enumerate(iBlockTrials):
            ## checking if this is still is an unassigned event
            if results.SimCount[iRow]!= 0:
                continue
            
            ## figuring out how many consecutive events of the same type can we have
            event_time= []
            for iConsecutiveEvent in range(2):
                if (iConsecutiveEvent+iTrial>=len(iBlockTrials)) or (results.Event[iRow]!=results.Event[iRow+iConsecutiveEvent]):
                    break
                event_time.append(results.Time[iRow+iConsecutiveEvent])
            
            ## trying for twos
            if len(event_time)>=2:
                dT= event_time[1]-event_time[0]
                iBracket= FOA[(FOA.ID==current_observer) & (FOA.Event==results.Event[iRow])].index
                if len(iBracket)>0 and dT>=FOA.MinRT[iBracket].values[0] and dT<=FOA.MaxRT[iBracket].values[0]:
                    # Match! Fill out and skip the rest of the loop
                    for iConsecutiveEvent in range(2):
                        results.set_value(iRow+iConsecutiveEvent, 'SimCount', 2)
                    continue
            
            ## single event, oh, well...
            results.set_value(iRow, 'SimCount', 1)
    print('\r', end= '')
results.to_csv(os.path.join(data_folder, 'Experiment1_Simultaneity.csv'), sep=';', index= False)    

E:\Software\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
